# DPO orca-math-korean

**Dataset:**
- https://huggingface.co/datasets/microsoft/orca-math-word-problems-200k
- https://huggingface.co/datasets/kuotient/orca-math-korean-dpo-pairs

**Model:**
- https://huggingface.co/soka0000/vclm-korean-7b

In [1]:
%pip install datasets transformers hf_transfer accelerate peft trl wandb scikit-learn - Uq


Note: you may need to restart the kernel to use updated packages.


## 데이터 준비

In [2]:
from datasets import load_dataset  # HuggingFace 데이터 셋 로드 함수

# DPO용 데이터 (chosen/rejected) 학습 데이터 로드
dataset = load_dataset("kuotient/orca-math-korean-dpo-pairs", split="train")
SAMPLE_SIZE = 10000
dataset = dataset.select(range(SAMPLE_SIZE))  # 앞에서부터 10000개만 선택
print(len(dataset))
print(dataset[100])


/opt/anaconda3/envs/nlp_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


10000
{'system': '당신은 인공지능 어시스턴트입니다.', 'question': '한 배럴에는 12리터(L)와 400밀리리터(ml)의 석유, B 배럴에는 7600밀리리터(ml)의 석유가 들어 있습니다. A 배럴과 B 배럴의 석유 양을 같게 하려면 A 배럴에서 B 배럴로 몇 리터(L)를 옮겨야 합니까?', 'chosen': '먼저 모든 측정값을 동일한 단위로 변환하여 계산을 쉽게 해봅시다. 모든 것을 밀리리터(ml)로 변환하겠습니다.\n\n배럴에는 12리터와 400밀리리터가 들어 있습니다. 1리터는 1000밀리리터와 같으므로 12리터를 밀리리터로 변환할 수 있습니다:\n12리터 = 12 * 1000밀리리터 = 12000밀리리터\n\n이제 이미 밀리리터 단위로 표시된 400밀리리터를 더합니다:\n12000밀리리터 + 400밀리리터 = 12400밀리리터\n\n따라서 배럴에는 총 12400밀리리터의 석유 가 들어 있습니다.\n\nB 배럴에는 7600밀리리터의 석유가 들어 있습니다.\n\n두 배럴의 석유 양을 동일하게 하려면 두 양의 평균을 구해야 합니다:\n두 배럴의 총량 = 12400밀리리터(A배럴) + 7600밀리리터(B배럴)\n두 배럴의 총량 = 20000밀리리터\n\n이제 이 총량을 2로 나누어 각 배럴에 해당하는 양을 구합니다:\n각 배럴의 동일한 양 = 20000밀리리터 / 2\n각 배럴의 동일한 양 = 10000밀리리터\n\n현재 A 배럴에는 12400밀리리터가 있으므로, 두 배럴의 양이 각각 10000밀리리터가 되도록 일부를 B 배럴로 이동해야 합니다.\n\nA에서 B로 이동할 양 = 12400밀리리터(A 배럴) - 10000밀리리터(동일한 양)\nA에서 B로 이동할 양 = 2400밀리리터\n\n따라서 두 배럴의 석유 양을 동일하게 하기 위해 A 배럴에서 B 배럴로 2400밀리리터(또는 2.4리터)를 이동해야 합니다.', 'rejected': ' 먼저 B 배럴의 석유 양을 리터로 변환하여 A 배럴의 석유 양과 비교해야 합니다.\n\nB 배럴의

In [3]:
print(dataset[9999])

{'system': '당신은 인공지능 어시스턴트입니다.', 'question': '제임스는 일주일 동안 500개의 깡통을 모아 깡통 집을 짓기로 결심합니다. 첫날에는 50개의 깡통을 모았습니다. 둘째 날에는 첫날에 모은 깡통의 특정 배수를 모았습니다. 셋째 날에는 둘째 날에 모은 깡통보다 50개를 더 모았습니다. 그는 일주일의 나머지 날에도 매일 50개씩 같은 수의 깡통을 모았습니다. 둘째 날에 모은 깡통 수와 첫째 날에 모은 깡통 수의 비율은 얼마입니까?', 'chosen': '두 번째 날에 제임스가 수집한 깡통의 수를 첫 번째 날에 수집한 깡통의 수에 \\( x \\) 배로 표시해 봅시다. 따라서 둘째 날에 그는 \\( 50x \\) 깡통을 수집합니다.\n\n셋째 날에는 \\( 50x - 50 \\) 깡통을 수집합니다.\n\n남은 4일(4일부터 7일까지) 동안 그는 매일 50개의 깡통을 수집하여 총 4 \\times 50 = 200 개의 깡통을 수집합니다.\n\n한 주 동안 수집한 깡통의 총 개수는 매일 수집한 깡통의 합으로, 500개의 깡통과 같아야 합니다. 따라서 우리는 다음과 같습니다:\n\n\\[ 50 + 50x + (50x - 50) + 200 = 500 \\]\n\n유사한 항을 결합하면 다음과 같습니다:\n\n\\[ 50x + 50x + 200 = 500 \\]\n\n\\[ 100x + 200 = 500 \\]\n\n양쪽에서 200을 빼면 다음과 같습니다:\n\n\\[ 100x = 300 \\]\n\n양변을 100으로 나누면 다음과 같습니다:\n\n\\[ x = 3 \\]\n\n따라서 둘째 날에 수집한 깡통의 수와 첫째 날에 수집한 깡통의 수의 비율은 \\( x:1, 즉 3:1입니다.', 'rejected': ' 둘째 날에 제임스가 모은 깡통의 수를 \\( x \\)`로 표시해 봅시다.\n\n첫째 날에는 50개의 깡통을 모았습니다.\n둘째 날에는 \\( x \\)개의 깡통을 모았습니다.\n셋째 날에는 \\( x + 50 \\)개의 깡통을 모았습니

# 모델 준비

https://huggingface.co/soka0000/vclm-korean-7b

In [4]:
# Policy(학습 대상) 모델/ 토크나이저 로드

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer  # 생성형 모델

model_name = "soka0000/vclm-korean-7b"  # 모델 이름

# 토크나이저 로드, trust_remote_code = 커스텀 코드 허용
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.bfloat16,  # 메모리 절약 및 연산 효율성을 높인다.
    device_map="auto",  # CPU/GPU 자동배치해준다.
    trust_remote_code=True,  # 모델 저장소에 있는 커스텀 코드 허용
)


Loading weights: 100%|██████████| 339/339 [00:03<00:00, 108.84it/s, Materializing param=model.norm.weight]                              
Some parameters are on the meta device because they were offloaded to the disk.


# 데이터 전처리

In [5]:
# 원본 샘플(system/question/chosen/rejected) -> prompt/chosen/rejected 형태의 모델용 학습 데이터 형태로 바꾸는 함수


def preprocess_dpo_data(example):
    messages = [
        {"role": "system", "content": example["system"]},  # 시스템 프롬프트
        {"role": "user", "content": example["question"]},  # 사용자 프롬프트
    ]

    # 모델이 기대하는 포멧으로 변환 (|im_start|system...|im_start|user...)  답변 생성 시작 지점까지 모두 포함
    prompt_style = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    # DPOTrainer가 사용하기 좋은 형태
    return {
        "prompt": prompt_style,  # 템플릿이 적용된 프롬프트
        "chosen": example["chosen"],  # 선호 응답
        "rejected": example["rejected"],  # 비선호 응답
    }


dataset_preprocessed = dataset.map(
    preprocess_dpo_data
)  # 전체 데이터셋에 전처리 함수 적용

print(f"before : {dataset.column_names}")
print(f"after : {dataset_preprocessed.column_names}")  # prompt 컬럼은 추가 생성


Map: 100%|██████████| 10000/10000 [00:00<00:00, 16084.60 examples/s]

before : ['system', 'question', 'chosen', 'rejected']
after : ['system', 'question', 'chosen', 'rejected', 'prompt']


In [6]:
dataset_preprocessed["prompt"][0]


'<|im_start|>system\n당신은 인공지능 어시스턴트입니다.<|im_end|>\n<|im_start|>user\n정국이 5위입니다. 정국보다 결승선을 먼저 통과한 사람의 수를 찾아보세요.<|im_end|>\n<|im_start|>assistant\n'

In [7]:
# 데이터셋 분할
# - train/val/test 비율을 8:1:1

train_size = int(len(dataset_preprocessed) * 0.8)
val_size = int(len(dataset_preprocessed) * 0.1)
test_size = int(len(dataset_preprocessed) * 0.1)
train_dataset = dataset_preprocessed.select(range(train_size))
val_dataset = dataset_preprocessed.select(range(train_size, train_size + val_size))
test_dataset = dataset_preprocessed.select(
    range(train_size + val_size, train_size + val_size + test_size)
)

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))


8000
1000
1000


In [8]:
# 모델로부터 텍스트 응답을 생성하는 함수
def generate_response(model, tokenizer, quesion):
    prompt = quesion  # 입력 질문을 그대로 프롬프트로 사용

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    # 텍스트를 토큰화해서 PyTorch 텐서로 변환 후, 모델이 있는 디바이스(CUDA/MPS/CPU)로 이동

    with torch.no_grad():  # 추론 모드 (그래디언트 계산 비활성화 → 메모리/속도 절약)
        outputs = model.generate(
            **inputs,  # input_ids, attention_mask 등 전달
            max_length=1024,  # 생성 포함 전체 최대 토큰 길이
            do_sample=True,  # 샘플링 방식 사용 (확률 기반 생성)
            top_k=50,  # 상위 50개 후보 중에서만 샘플링
            top_p=0.95,  # 누적 확률 95% 안에서만 샘플링 (nucleus sampling)
            temperature=0.5,  # 낮을수록 더 보수적/결정적 생성 창의성 0.5
            num_return_sequences=1,  # 생성 결과 1개 반환
            eos_token_id=tokenizer.eos_token_id,  # 종료 토큰 설정
            pad_token_id=tokenizer.pad_token_id  # 패딩 토큰 설정
        )

        generate_text = tokenizer.decode(outputs[0])
        # 생성된 토큰 시퀀스를 문자열로 디코딩

        return generate_text.replace(prompt, "").strip()
        # 프롬프트 부분 제거 후, 생성된 답변만 반환


In [ ]:
test_subset = test_dataset.select(range(3))

for i, example in enumerate(test_subset):  # 샘플 3개를 순회
    quesiton = example["prompt"]
    answer = example["chosen"]  # 선호 답변(데이터셋에서는 정답)

    print(f"[질문]:\n{quesiton}")
    print(f"\n[정답]:\n{answer}")

    generated_answer = generate_response(model, tokenizer, quesiton)
    print(f"\n[모델 생성 답변]:\n{generated_answer}")
    print("-" * 100)


[질문]:
<|im_start|>system
당신은 인공지능 어시스턴트입니다.<|im_end|>
<|im_start|>user
알리사와 아비게일은 과학 프로젝트를 위해 빈 캔 100개를 모아야 합니다. 오늘 현재 알리사는 빈 캔을 몇 개 모았고, 아비게일은 빈 캔 43개를 모았습니다. 그들은 빈 캔을 27개 더 모아야 합니다. 지금까지 알리사가 모은 빈 캔은 몇 개인가요?<|im_end|>
<|im_start|>assistant


[정답]:
알리사가 얼마나 많은 빈 캔을 모았는지 알아내려면, 아비게일이 모은 캔의 수와 프로젝트에 필요한 총 캔 수에서 아직 모아야 할 캔의 수를 빼야 합니다.

알리사의 캔 + 아비게일의 캔 + 아직 필요한 캔 = 필요한 총 캔 수입니다.
알리사의 캔 = 필요한 총 캔 수 - (아비게일의 캔 + 아직 필요한 캔)

우리는 이것을 알고 있습니다:
필요한 총 캔 수 = 100
아비게일의 캔 = 43
아직 필요한 캔 = 27

이제 숫자를 연결할 수 있습니다:

알리사의 캔 = 100 - (43 + 27)
알리사의 캔 = 100 - 70
알리사의 캔 = 30

알리사는 지금까지 빈 캔 30개를 모았습니다.

[모델 생성 답변]:
현재까지 알리사가 모은 빈캔의 수를 구하려면, 총 필요한 빈캔의 수에서 아비게일이 모은 빈캔의 수를 빼고, 그 결과에서 아비게일이 더해야 하는 빈캔의 수를 빼야 해요. 하지만 문제는 두 사람이 빈캔을 함께 모으고 있다는 점을 고려해야 해요. 따라서, 알리사가 아비게일보다 더 많은 빈캔을 가져야 하므로, 먼저 아비게일이 더해야 하는 빈캔의 수를 빼야 해요. 아비게일이 더해야 하는 빈캔의 수: 27개 아비게일이 가져야 하는 빈캔의 수: 43개 이제, 두 사람 모두 더해야 하는 빈캔의 수를 빼요: 43 - 27 = 16개 이제, 이 값에서 알리사가 더해야 하는 빈캔의 수를 빼요: 16 - 27 = -11개 이 결과는 불가능해 보여요. 문제는 두 사람이 빈캔을 함께 모으고 있다는 점을 고려하지 않았다는 것을 의미해요

# DPO 학습준비

In [ ]:
# LoRA(PEFT) 적용으로 정책모델 학습 파라미터 축소 확인
from peft import LoraConfig, get_peft_model  # LoRA 설정 / PEFT 적용함수

lora_config = LoraConfig(
    r=8,  # LoRA 랭크(추가로 학습할 저차원 행렬 크기)
    lora_alpha=16,  # LoRA 스케일 (업데이트 강도 조절)
    target_modules=["q_proj", "v_proj"],  # LoRA에 붙일 모듈(어텐션의 Q/V 프로젝션)
    lora_dropout=0.05,  # 과적합 완화
    bias="none",  # Bias 학습 제외
    task_type="CAUSAL_LM",  # 작업 유형(디코더 LM)
)

# 정책 모델(학습 대상)
model = get_peft_model(model, lora_config)  # 기존 모델에 LoRA 적용 (PEFT 모델)

model.print_trainable_parameters()  # 학습 가능한 파라미터 수/비율 체크!


In [ ]:
# 참조모델 : 정책모델이 얼마나 바뀌었는지 비교 기준이 되는 고정 모델
# - chosen/rejected 답변에 대한 생성 확률 비교값 제공


# 참조모델 (학습하지 않은 기본모델)
reference_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.bfloat16,  # 메모리 절약 및 연산 효율성을 높인다.
    device_map="auto",  # CPU/GPU 자동배치해준다.
    trust_remote_code=True,  # 모델 저장소에 있는 커스텀 모델 코드 허용
)


# 학습방지 처리
reference_model.eval()  # 평가모드 전환
for param in reference_model.parameters():  # 모든 파라미터 순회
    param.requires_grad = False  # gradient 계산 / 업데이트 방지


In [ ]:
# DPOConfig 설정 및 DPOTrainer로 학습 실행
from trl import DPOTrainer, DPOConfig

hub_model_id = (
    "SEUNGHUN12/vclm-korean-7b-orca-math-korean-dpo"  # 학습 완료 모델을 올릴 hub 저장소
)


# DPO 학습 하이퍼파라미터/로깅/저장 설정
training_args = DPOConfig(
    output_dir="vclm-korean-7b-orca-math-korean-dpo",  # 체크포인트/로그 저장 폴더
    num_train_epochs=1,  # 전체 데이터를 1번 반복 학습
    per_device_train_batch_size=2,  # GPU 1개당 배치 크기
    # 4번 누적 후 업데이트(실제 배치 효과: 2*4=8)
    gradient_accumulation_steps=4,
    learning_rate=5e-5,  # 학습률
    eval_strategy="steps",  # 일정 step마다 평가 수행
    save_strategy="steps",  # 일정 step마다 저장 수행
    logging_steps=50,  # 50 step마다 학습 로그 출력
    # fp16 비활성화(여기서는 bf16 사용)
    fp16=False,
    # bfloat16 사용(지원 GPU에서 안정적/빠름)
    bf16=True,
    # Ampere 이상에서 matmul 가속 옵션(정밀도 약간 완화)
    tf32=True,
    beta=0.1,  # DPO의 beta(선호 강도 조절)
    max_length=512,  # prompt+답변을 포함한 최대 길이
    max_prompt_length=256,  # prompt에 할당할 최대 길이
    remove_unused_columns=False,  # DPO에 필요한 컬럼이 제거되지 않도록 유지
    push_to_hub=True,  # 학습 결과를 Hugging Face Hub로 업로드
    hub_model_id=hub_model_id,  # 업로드할 저장소 이름
    hub_strategy="end",  # 학습 끝난 뒤 한 번만 업로드
    report_to=["wandb"],  # wandb로 학습 로그 전송
)


In [ ]:
# DPO Trainer 생성(정책모델 vs 참조모델 비교 학습)
dpo_trainer = DPOTrainer(
    model=model,  # 정책모델
    ref_model=reference_model,  # 참조모델
    args=training_args,  # 학습 설정
    train_dataset=train_dataset,  # 학습 데이터 (prompt/chosen/rejected)
    eval_dataset=val_dataset,  # 검증 데이터
    processing_class=tokenizer,  # 토큰화 처리
)

dpo_trainer.train()  # DPO 학습 실행


In [ ]:
# 학습이 중간에 끊어졌거나 혹은 자동으로 업로드되지 않은 경우
# dpo_trainer.push_to_hub('Commit!') # 현재 학습된 체크포인트/모델을 Hub에 커밋 메시지와 함께 업로드

# DPO Model 테스트

In [ ]:
test_subset = test_dataset.select(range(3))

for i, example in enumerate(test_subset):  # 샘플 3개를 순회
    quesiton = example["prompt"]
    answer = example["chosen"]  # 선호 답변(데이터셋에서는 정답)

    print(f"[질문]:\n{quesiton}")
    print(f"\n[정답]:\n{answer}")

    generated_answer = generate_response(model, tokenizer, quesiton)
    print(f"\n[모델 생성 답변]:\n{generated_answer}")
    print("-" * 100)


## DPO의 성능평가

In [ ]:
import torch
import torch.nn.functional as F

# prompt에 대해 response가 얼마나 그럴듯한지(평균 log_prob) 계산하는 함수


def calculate_log_prob(model, tokenizer, prompt, response):
    """
    주어진 prompt에 대한 response의 log probability(평균)를 계산합니다.

    Args:
        model: HuggingFace AutoModelForCausalLM (or similar)
        tokenizer: HuggingFace AutoTokenizer
        prompt (str): 프롬프트 텍스트
        response (str): 응답 텍스트

    Returns:
        float: response 토큰들의 평균 log probability
    """
    full_text = prompt + " " + response  # Prompt + 답변
    inputs = tokenizer(
        full_text,
        return_tensors="pt",
        truncation=True,  # 최대 글자 길이 이상시 자름
        max_length=512,  # 최대 토큰 길이
    ).to(model.device)

    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    prompt_tokens = tokenizer(
        prompt, return_tensors="pt", truncation=True, max_length=512
    ).to(model.device)
    prompt_len = prompt_tokens["input_ids"].shape[1]

    # 추론 모드(그래프 미생성/ 계산만 진행) : 메모리가 절약
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = logits[..., 1:].contiguous()

    log_probs = F.log_softmax(shift_logits, dim=-1)

    true_log_probs = torch.gather(log_probs, 2, shift_labels.unsqueeze(-1)).squeeze(-1)

    seq_len = shift_labels.shape[1]
    if prompt_len >= seq_len:
        valid_log_probs = true_log_probs[:, -1:]
    else:
        start_idx = max(0, prompt_len - 1)
        valid_log_probs = true_log_probs[:, start_idx:]

    avg_log_probs = valid_log_probs.mean().item()

    return avg_log_probs


In [ ]:
def calcualte_preference_accuracy(model, tokenizer, dataset, num_samples=100):
    """
    모델이 선호(chosen) 답변에 비선호(rejected) 답변 보다 더 높은 확률을 부여하는지를 평가
    """

    correct = 0
    total = min(num_samples, len(dataset))

    print(f"{total}개 샘플에 대해 정확도 계산 시작")

    model.eval()
    for idx in range(total):
        example = dataset[idx]

        prompt = example["prompt"]
        chosen = example["chosen"]
        rejected = example["rejected"]

        chosen_score = calculate_log_prob(model, tokenizer, prompt, chosen)
        rejected_score = calculate_log_prob(model, tokenizer, prompt, rejected)

        if chosen_score > rejected_score:
            correct += 1

        if (idx + 1) % 10 == 0:
            print(
                f" - 진행률 : {idx+1}{total} (현재 정확도: {correct/(idx+1)*100:.2f}%)"
            )

    accuracy = correct / total
    return accuracy


test_accuracy = calcualte_preference_accuracy(
    model, tokenizer, dataset, num_samples=300
)
print(test_accuracy)
